## Netflix Data Visualisation and recomendation system ##
$
\\Netflix
$

In [ ]:
# Importing required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Importing Netflix Dataset

df=df = pd.read_csv("/kaggle/input/netflix-shows/netflix_titles.csv")
df.head()

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df.dtypes

In [ ]:
#As we can see 'release_year' is string type, we need to convert this column into date type
df.date_added=pd.to_datetime(df.date_added )
df.head()

In [ ]:
df.dtypes

In [ ]:
print(df.show_id.isnull().value_counts())
print(df.type.isnull().value_counts())
print(df.title.isnull().value_counts())
print(df.director.isnull().value_counts())
print(df.cast.isnull().value_counts())
print(df.country.isnull().value_counts())
print(df.date_added.isnull().value_counts())
print(df.release_year.isnull().value_counts())
print(df.rating.isnull().value_counts())
print(df.duration.isnull().value_counts())
print(df.listed_in.isnull().value_counts())
print(df.description.isnull().value_counts())

In [ ]:
df.isnull().sum()

 According to above description 'director':2389, 'cast':718, 'country':507, 'date_added':10, 'rating':7 has null value. We need to remove this null value or fill with corresponding values. 

In [ ]:
#Visualize the null values through heat map
!pip install missingno


In [ ]:
import seaborn as sns
import missingno as ms

In [ ]:
heatmap_null_values=ms.heatmap(df, figsize=(10,7), cmap='PuBuGn')

In [ ]:
#count of unique values
unique_val=df.nunique()
unique_val

In [ ]:
#Null values of corresponding rows
df[df.rating.isna()]

In [ ]:
#Fill 'ratings' NaN values with specific ratings
user_rating={67:'TV-MA', 2359:'R',3660:'R', 3736:'PG-13', 3737:'TV-MA', 3738:'R', 4323:'PG-13'}

In [ ]:
df.rating=df.rating.fillna(user_rating)

In [ ]:
df.rating.isnull().value_counts()

In [ ]:
df[df.rating.isna()] # No null values are present in rating column

In [ ]:
#Replacing country null value with maximum number of country
country=df.loc[df.country.notnull(), 'country'].astype('str').apply(lambda t: t.split(', '))

country=list(country)
len(country)

In [ ]:
!pip install mlxtend

In [ ]:
from mlxtend.frequent_patterns import apriori
from mlxtend.preprocessing import TransactionEncoder

Apriori is a popular algorithm for extracting frequent itemsets with applications in association rule learning. As we can see in 'country' cloumn frequent names are given, thats'y I have used apriori algorithm for finding frequency distribution

In [ ]:
#Initiating encoder and fit and transform the encoder
encoder_model=TransactionEncoder().fit(country)

encode_country=encoder_model.transform(country)

In [ ]:
#Creating new dataframe with encoded counties
df_encode=pd.DataFrame(encode_country, columns=encoder_model.columns_, index=df.loc[df.country.notnull(), 'show_id'])
df_encode.head()

In [ ]:
country_share=df_encode.mean().sort_values(ascending=False)

country_share=country_share * 100
country_share

In [ ]:

import matplotlib as mpl

mpl.style.use('ggplot') # optional: for ggplot-like style

# check for latest version of Matplotlib
print('Matplotlib version: ', mpl.__version__) # >= 2.0.0

In [ ]:
#Visualiztion of country share in pie chart

# take countries that share more than 1%
country_share=country_share[country_share > 1 ]
labels=country_share.round(3).astype('str') + '%'
#explode_list = [0.1, 0, 0, 0, 0.1, 0.1]
country_share.plot(kind='pie', figsize=(10,10), labels=labels, shadow=True)

plt.title('Percent of produced Movies/TV Show by Country', fontsize=20)
plt.legend(labels=country_share.index, loc='upper left')
plt.show()

In [ ]:
#Another method
# take countries that share more than 2%
country_share_1 = country_share[country_share > 2]
labels = country_share_1.round(3).astype('str') + ' %'

fig1, ax1 = plt.subplots(figsize=(10,10), facecolor='white')
ax1.pie(country_share_1, labels=labels, labeldistance=1.05,
        shadow=True)
plt.title('Percent of produced Movies/TV Show by Country', fontsize=20)
plt.legend(labels=country_share_1.index, loc='upper right')
plt.show()


'Unites States' has the maximum frequency. I will fill 'NaN' value in country column with 'United States'

In [ ]:
df['country']=df['country'].fillna('United States')
df.isna().sum()

In [ ]:
#Drop all the null values from cast column
df=df.dropna(subset=['cast'], how='all')

In [ ]:
df.cast.isnull().value_counts() 

No missing value is present in 'cast' column

In [ ]:
df.isnull().sum()

In [ ]:
country_share

In [ ]:
#Formation of dataframe from 'country_share' series. 
df_country_share=country_share.to_frame().reset_index()
df_country_share.head()

In [ ]:
df_country_share=df_country_share.rename(columns={'index':'Country', 0:'Frequency'})

In [ ]:
df_country_share.head()

In [ ]:
# Visualise data in World Map
import folium
print('Folium installed and imported')

In [ ]:
world_map = folium.Map()
world_map

In [ ]:
!pip install xlrd

In [ ]:
# download countries geojson file
!wget --quiet https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/world_countries.json
    
print('GeoJSON file downloaded!')

In [ ]:
world_json= '/kaggle/input/jsonfile/world_countries.json.1'
world_json

In [ ]:
# generate choropleth map using Percent of produced Movies/TV Show of each country
world_map.choropleth(
    geo_data=world_json,
    data=df_country_share,
    columns=['Country', 'Frequency'],
    key_on='feature.properties.name',
    fill_color='RdBu',
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='Percent of produced Movies/TV Show by Country'
)

# display map
world_map

In [ ]:
df.isnull().sum()

In [ ]:
df.head()

In [ ]:
#Extraction of genre fron 'listed_in' column
genre=df['listed_in'].apply(lambda x:x.strip() )
genre=genre.str.split(', ')
#another method:
#genre=df['listed_in'].apply(lambda x:x.split(', ') )
genre
genre_list=list(genre)
print(genre_list[5])
print(len(genre_list))



Initiate encoder for finding count of unique items

In [ ]:
#Initiate encoder
genre_encoder=TransactionEncoder().fit(genre)
transform=genre_encoder.transform(genre)


In [ ]:
df_genre=pd.DataFrame(transform, columns=genre_encoder.columns_, index=df['show_id'])

In [ ]:
df_genre.shape

In [ ]:
#Total number of counts of each genre type
genre_count=df_genre.sum().sort_values(ascending=False)
genre_count.head(10)

In [ ]:
#creating Bar plot for genres

genre_count.plot(kind='bar', figsize=(15,7), color='b')

plt.xlabel('Genre Type')
plt.ylabel('Count of Contents')
plt.title('Plot between Genre Type vs Count of Contents')
plt.show()

In [ ]:
movies=df[df['type']=='Movie']['release_year'].value_counts().rename('Movies').reset_index()
TV_shows=df[df['type']=='TV Show']['release_year'].value_counts().rename('TV Shows').reset_index()

#Sorting value counts by years
movies=movies.sort_values(by='index')
TV_shows=TV_shows.sort_values(by='index')

movies_plot=movies.plot(kind='line', x='index', y='Movies', legend='Movies', color='r')

TV_Show_plot=TV_shows.plot(kind='line', x='index', y='TV Shows', legend='TV Shows', color='g')

## Recomendation System ##

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import sigmoid_kernel
model=TfidfVectorizer(max_df=2, min_df=1, token_pattern='(?u)\\b\\w\\w+\\b', max_features=None, stop_words='english', ngram_range=(1, 4))

In [ ]:
df['combination']= df['description'] + df['cast'] + df['director']
df.head(2)

In [ ]:
df['combination']=df['combination'].fillna(' ')

In [ ]:
# Utilising fit and transform method on model object
tf_matrix=model.fit_transform(df['combination'])

sigmoid=sigmoid_kernel(tf_matrix, tf_matrix)
sigmoid[1]

In [ ]:
indices=pd.Series(df.index, index=df['title'].drop_duplicates())
indices

In [ ]:
def recommend(title,sig=sigmoid):
    idx = indices[title]
    sig_scores = list(enumerate(sig[idx]))
    sig_scores = sorted(sig_scores,key = lambda x:x[1], reverse = True)
    sig_scores = sig_scores[1:11]
    movies_indices = [i[0] for i in sig_scores]
    return df['title'].iloc[movies_indices]

In [ ]:
recommend("Inside Man: Most Wanted")

## Please drop your suggetions here for any kind of modification. Your suggestion will be appreciated toward more learning ##